In [7]:
import pandas as pd
import plotly.graph_objs as go 
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from tqdm import tqdm 
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Import des données

In [8]:
apple = pd.read_csv("../data/AAPL.csv")
amazon = pd.read_csv("../data/AMZN.csv")
berkshire = pd.read_csv("../data/BRK-B.csv")
google = pd.read_csv("../data/GOOG.csv")
meta = pd.read_csv("../data/META.csv")
microsoft = pd.read_csv("../data/MSFT.csv")
nvidia = pd.read_csv("../data/NVDA.csv")
tesla = pd.read_csv("../data/TSLA.csv")
unitedhealth = pd.read_csv("../data/UNH.csv")
exxon = pd.read_csv("../data/XOM.csv")

In [9]:
titres = {"Apple" : apple, 
          "Amazon" : amazon,
          "Berkshire" : berkshire,
          "Google" : google,
          "Meta" : meta,
          "Microsoft" : microsoft,
          "Nvidia" : nvidia,
          "Tesla" : tesla,
          "UnitedHealth" : unitedhealth,
          "Exxon" : exxon}

# Calcul des prédictions

In [10]:
def stock_volatility_forecast(stock):
    # stock['Date'] = pd.to_datetime(stock['Date'])
    # stock['Volatility_HighLow_10days'] = stock['High'].rolling(window=10).std() / stock['Low'].rolling(window=10).mean()

    stock['Return'] = stock['Close'].pct_change()
    # stock = stock.dropna()
    # stock = stock.drop(columns=["Open", 'Adj Close'])

    # Calcul de la volatilité sur une fenêtre mobile
    stock['Volatility'] = stock['Return'].rolling(window=10).std()

    # Définition de la cible comme la volatilité future
    stock['Target'] = stock['Volatility'].shift(-10)
    stock['Date'] = pd.to_datetime(stock['Date'])
    stock['Adj Close'] = stock['Adj Close'].pct_change()
    stock['High'] = stock['High'].pct_change()
    stock['Low'] = stock['Low'].pct_change()
    stock['Volume'] = stock['Volume'].pct_change()
    stock['dailyChange'] = (stock['Adj Close'] - stock['Open']) / stock['Open']

    # stock['Volatility10'] = stock['Adj Close'].rolling(window=10).std()
    # stock['VolMA10'] = stock['Volume'].rolling(window=10).mean()

    # stock['priceDirection'] = (stock['Adj Close'].shift(-1) - stock['Adj Close'])
    rsi_period = 10
    chg = stock['Adj Close'].diff(1)
    gain = chg.mask(chg<0,0)
    loss = chg.mask(chg>0,0)
    avg_gain = gain.ewm(com = rsi_period - 1, min_periods = rsi_period).mean()
    avg_loss = loss.ewm(com = rsi_period - 1, min_periods = rsi_period).mean()
    rs = abs(avg_gain/avg_loss)
    stock['rsi'] = 100-(100/(1+ rs))
    stock['Std_dev']=  stock['Adj Close'].rolling(10).std()

    stock['Williams%R'] = (stock['High'].max() - stock['Adj Close'])/(stock['High'] - stock['Low'].min()) * -100
    stock = stock.replace([np.inf, -np.inf], np.nan)
    stock = stock.drop(columns=["Open", 'Adj Close'])

    stock.dropna(inplace=True)

    stock = stock.dropna()

    window_size = 90
    step_size = 7
    prediction_days = 7

    prediction_dates = []
    predictions = []
    for start in tqdm(range(0, len(stock) - window_size - prediction_days, step_size)):
        end = start + window_size
        train = stock.iloc[start:end]
        test = stock.iloc[end:end + prediction_days]

        model = XGBRegressor(n_estimators=400, max_depth=35, learning_rate=0.2, verbosity=0)
        model.fit(train.drop(['Date', 'Target', 'Volume'], axis=1), train['Target'])

        pred = model.predict(test.drop(['Date', 'Target', 'Volume'], axis=1))
        predictions.append(pred)

        prediction_date = stock.iloc[end + prediction_days - 1]['Date']
        prediction_dates.append(prediction_date)

    predictions = [value[6] for value in predictions]

    df_predictions = pd.DataFrame({
        'Date': prediction_dates,
        'Prediction': predictions
    })

    merged_df = stock.merge(df_predictions, on='Date', how='inner')

    merged_df = merged_df.drop(columns = ["Target"])

    return(merged_df)


In [11]:
predictions_dic = {}

In [12]:
for key in titres :
    predictions_dic[key] = stock_volatility_forecast(titres[key])

  0%|          | 0/379 [00:00<?, ?it/s]

100%|██████████| 379/379 [00:51<00:00,  7.32it/s]


In [13]:
predictions_dic

{'Apple':           Date      High       Low       Close    Volume    Return  \
 0   2013-06-06 -0.008253 -0.021771   15.659286  0.434786 -0.014940   
 1   2013-06-17 -0.001352  0.004341   15.428571 -0.045801  0.004534   
 2   2013-06-26 -0.007357 -0.007948   14.216786  0.170489 -0.011326   
 3   2013-07-08 -0.005410 -0.011316   14.823214  0.087998 -0.005678   
 4   2013-07-17  0.003506  0.009548   15.368214 -0.081037  0.000256   
 ..         ...       ...       ...         ...       ...       ...   
 374 2023-10-30  0.013080  0.012228  170.289993 -0.125952  0.012305   
 375 2023-11-08  0.005536  0.014639  182.889999 -0.300435  0.005885   
 376 2023-11-17 -0.003037 -0.000424  189.690002 -0.064143 -0.000105   
 377 2023-11-29  0.005286 -0.002270  189.369995  0.119712 -0.005410   
 378 2023-12-08  0.005077  0.000413  195.710007  0.124260  0.007412   
 
      Volatility  dailyChange        rsi   Std_dev  Williams%R  Prediction  
 0      0.009082    -1.000939  43.840346  0.009082  -99.2216

In [14]:
dfs_to_concat = []

for name, df in predictions_dic.items():
    df['Date'] = pd.to_datetime(df['Date'])
    dfs_to_concat.append(df[['Date', 'Prediction', "Volatility"]])

portfolio_predictions = pd.concat(dfs_to_concat)
portfolio_avg_predictions = portfolio_predictions.groupby('Date').mean().reset_index()

print(portfolio_avg_predictions.head())

        Date  Prediction  Volatility
0 2013-06-06    0.019148    0.017101
1 2013-06-17    0.017013    0.014440
2 2013-06-26    0.016902    0.014275
3 2013-07-08    0.016722    0.012786
4 2013-07-17    0.018768    0.014141


In [15]:
fig = go.Figure()

# Ajout de la série financière
fig.add_trace(go.Scatter(x = portfolio_avg_predictions.Date, 
                         y = portfolio_avg_predictions['Prediction'],
                         mode = 'lines',
                         name ='Prédiction', 
                         fill = "tozeroy"))

fig.add_trace(go.Scatter(x = portfolio_avg_predictions.Date, 
                         y = portfolio_avg_predictions['Volatility'],
                         mode = 'lines',
                         name ='Réel', 
                         fill = "tozeroy"))

# Mise à jour des titres
fig.update_layout(title="Évolution de la volatilité du ptf",
                  xaxis_title='Date',
                  yaxis_title='Valeur')

# Affichage du graphique
fig.show()